In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
# from keras.layers.convolutional import UpSampling2D, Conv2D
from tensorflow.keras.layers import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras. optimizers import Adam, SGD
import matplotlib.pyplot as plt
import sys
import numpy as np


In [4]:
num_rows = 28
num_cols = 28
num_channels = 1
input_shape = (num_rows, num_cols, num_channels)
z_size = 100
batch_size = 128

(train_ims, _), (_, _) = mnist.load_data()
train_ims = train_ims / 127.5 - 1.
train_ims = np.expand_dims(train_ims, axis=3)

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

Defining the Generator

In [5]:
def build_generator():
    gen_model = Sequential()
    gen_model.add(Dense(256, input_dim=z_size))
    gen_model.add(LeakyReLU(alpha=0.2))
    gen_model.add(BatchNormalization(momentum=0.8))
    gen_model.add(Dense(512))
    gen_model.add(LeakyReLU(alpha=0.2))
    gen_model.add(BatchNormalization(momentum=0.8))
    gen_model.add(Dense(1024))
    gen_model.add(LeakyReLU(alpha=0.2))
    gen_model.add(BatchNormalization(momentum=0.8))
    gen_model.add(Dense(np.prod(input_shape), activation='tanh'))
    gen_model.add(Reshape(input_shape))

    gen_noise = Input(shape=(z_size,))
    gen_img = gen_model(gen_noise)
    return Model(gen_noise, gen_img)


# Defining the Discriminator

In [6]:
def build_discriminator():

	disc_model = Sequential()
	disc_model.add(Flatten(input_shape=input_shape))
	disc_model.add(Dense(512))
	disc_model.add(LeakyReLU(alpha=0.2))
	disc_model.add(Dense(256))
	disc_model.add(LeakyReLU(alpha=0.2))
	disc_model.add(Dense(1, activation='sigmoid'))

	disc_img = Input(shape=input_shape)
	validity = disc_model(disc_img)
	return Model(disc_img, validity)


# Computing the Loss Function

In [9]:
# discriminator
disc= build_discriminator()
disc.compile(loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'])

z = Input(shape=(z_size,))
# generator
img = build_generator()(z)

disc.trainable = False

validity = disc(img)

# combined model
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer='sgd')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Model  & Optimizing the Loss

In [12]:
def intialize_model():
    disc= build_discriminator()
    disc.compile(loss='binary_crossentropy',
        optimizer='sgd',
        metrics=['accuracy'])

    generator = build_generator()

    z = Input(shape=(z_size,))
    img = generator(z)

    disc.trainable = False

    validity = disc(img)

    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer='sgd')
    return disc, generator,combined


# Define Training

In [19]:
def train(epochs, batch_size=128, sample_interval=50):
    # load images
    (train_ims, _), (_, _) = mnist.load_data()
    # preprocess
    train_ims = train_ims / 127.5 - 1.
    train_ims = np.expand_dims(train_ims, axis=3)

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    # training loop
    for epoch in range(epochs):

        batch_index = np.random.randint(0, train_ims.shape[0], batch_size)
        imgs = train_ims[batch_index]

        # create noise
        noise = np.random.normal(0, 1, (batch_size, z_size))
        # predict using the Generator
        gen_imgs = gen.predict(noise)

        # calculate loss functions
        real_disc_loss = disc.train_on_batch(imgs, valid)
        fake_disc_loss = disc.train_on_batch(gen_imgs, fake)
        disc_loss_total = 0.5 * np.add(real_disc_loss, fake_disc_loss)

        noise = np.random.normal(0, 1, (batch_size, z_size))

        g_loss = full_model.train_on_batch(noise, valid)

        # show progress
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
            epoch,
            disc_loss_total[0],  # Access loss from list
            100 * disc_loss_total[1],  # Access accuracy
            g_loss[0]  # Access generator loss
        ))

        # save outputs every few epochs
        if epoch % sample_interval == 0:
            one_batch(epoch)


# Generating Handwritten Digits

In [22]:
import os

def one_batch(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, z_size))
    gen_imgs = gen.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Create directory if it doesn't exist
    if not os.path.exists("images"):
        os.makedirs("images")

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(f"images/{epoch}.png")
    plt.close()


# Train

In [ ]:
disc, gen, full_model = intialize_model()
train(epochs=10000, batch_size=32, sample_interval=200)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
0 [D loss: 1.055802, acc.: 24.22%] [G loss: 0.915705]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1 [D loss: 0.943707, acc.: 38.41%] [G loss: 0.899524]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2 [D loss: 0.907686, acc.: 41.77%] [G loss: 0.882422]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
3 [D loss: 0.892690, acc.: 42.13%] [G loss: 0.875288]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4 [D loss: 0.895563, acc.: 40.90%] [G loss: 0.885378]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
5 [D loss: 0.896225, acc.: 39.89%] [G loss: 0.885032]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6 [D loss: 0.900667, acc.: 39.08%] [G loss: 0.891441]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
7 [D loss: 0.901878, acc.: 38.39%] [G loss: 0.894554]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
8 [D loss: 0.901917, acc.: 37.75%] [G loss: 0.895176]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
9 [D loss: 0.898155, acc.: 37.40%] [G loss: 0.890927]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
10 [D loss: 0.895564, acc.: 37.56%] [G loss: 0.890037]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
11